In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import fasttext
import re

In [2]:
fake = pd.read_csv('fake.csv')
fake.shape

(1339, 2)

In [3]:
non_fake = pd.read_csv('non-fake.csv')
non_fake = non_fake.sample(n=2000,random_state=10)

In [4]:
dataset = pd.concat([fake,non_fake])
dataset.sample(10)

,Review,Label
6038,সল্টগ্রিল বসুন্ধরা সিটি ...আমি চূড়ান্ত রকম হতা...,1
3252,ধানমন্ডি 15/a নম্বর এ আলমাস বিল্ডিং এর 3 তলায় ...,1
602,#The_MidtowN \r\n💥Combo pack 💥\r\n😍Offer Price...,0
730,ফার্মেসীতে ENO না পাওয়ায় MAXPRO দিয়েই দিলাম...,0
218,#positive... Review\r\nName: Tanu kitchen\r\np...,0
1189,#২.৫_কেজি_ওজনের_পিজ্জা\r\n#প্রতি_কামড়ে_কামড়ে_ম...,0
725,প্লেট ভর্তি খাবার মাত্র #১৩৯টাকায় আসলেই কি সম্...,0
7645,১০০টাকাতে দই ফুসকা আর ২০০টাকাতে ছিটা রুটি আর হ...,1
3744,পিজ্জাটি আসলে 12 inch ছিলো। রাস্তায় দাড়ায় খাওয়...,1
1334,#২_টাকায়_ডিপডিশপিজ্জা_খেলাম \r\nসদ্য চাকরি পাও...,0


In [5]:
dataset.to_csv('bangla_food_review.csv',index=False)

In [6]:
data = pd.read_csv('bangla_food_review.csv')
data.sample(5)

,Review,Label
1502,কাবাব সমাচার বর্তমানে ঢাকাশহরে সবচেয়ে বেশি যেই...,1
1150,১০ পিস চিকেনের সাথে আনলিমিটেড বিরিয়ানি অফার......,0
1656,#the_bitcoin_cafeকম বাজেটে bhalo khabar অথবা ট...,1
2260,Fully লোডেড রিভিউ ...প্রতি মাসের ৫ তারিখ নান্ন...,1
2418,কাল গেলাম Aangan এ murg.a.akbar আর গুলাব জামু...,1


In [7]:
data.dropna(inplace=True)
data.shape

(3339, 2)

In [8]:
data.Label.replace(1,"Not_Fake",inplace=True)
data.Label.replace(0,"Fake",inplace=True)
data.sample(5)

,Review,Label
2416,কি খবর জনগণ?ব্রাউনি খাবা নাকি? খেয়ে দেয়া টাইপ...,Not_Fake
2703,"কলকাতা কাচ্চি খাওয়ার শখ চাপলো, কিন্তু এই জ্যাম...",Not_Fake
1044,"Salads 🥒🥬🥦🌽 \r\n সালাদ খেয়ে পেট ভরে না, কে বলে...",Fake
1135,১/আপনি কি স্টুডেন্ট/চাকুরীজীবি???\r\n২/আপনি কি...,Fake
1775,মোহাম্মদপুর সলিমুল্লাহ রোডের একটু ভিতর দিকে ঢু...,Not_Fake


In [9]:
data["Label"] = "__label__" + data["Label"].astype(str)
data.sample(5)

,Review,Label
734,"Fuoco তে ২২ টাকার অফার দেখে চলে গেলাম , আর টেস...",__label__Fake
1468,সীফুড খেতে কক্সবাজার না গিয়ে উত্তরায় চলে আসুন।...,__label__Not_Fake
1575,#রেস্টুরেন্টে_যখন_ফুল_বাগান Time Square Dine ...,__label__Not_Fake
536,"""#DFSC_09\r\n#(Rupkotha /SERVICE NAME)\r\n**Pl...",__label__Fake
2819,প্রতিবছর চকবাজার ইফাতার বাজার যাওয়ার সুযোগ হতো...,__label__Not_Fake


In [10]:
data["category_description"] = data["Label"] +" "+ data["Review"]
data.sample(5)

,Review,Label,category_description
2969,খিলগাঁও এর ভিতর স্যান্ডুইচ এর মধ্যে Bundose বে...,__label__Not_Fake,__label__Not_Fake খিলগাঁও এর ভিতর স্যান্ডুইচ এ...
1046,সিঙাড়া হচ্ছে ইল্যুমিনাতি বা দাজ্জালের খাবার।🙏\...,__label__Fake,__label__Fake সিঙাড়া হচ্ছে ইল্যুমিনাতি বা দাজ্...
2459,প্লেসঃ আব্দুল মজিদ স্পেশাল ঘরোয়া খাবারFoodBank...,__label__Not_Fake,__label__Not_Fake প্লেসঃ আব্দুল মজিদ স্পেশাল ঘ...
429,*খাবার : কাপল কম্বো -৬৭৯/\r\n* ফ্রাইড রাইস ...,__label__Fake,__label__Fake *খাবার : কাপল কম্বো -৬৭৯/\r\n* ফ...
1771,অবশেষে আমরাও খেয়ে আসলাম Pizzaology Wari এর ১৫০...,__label__Not_Fake,__label__Not_Fake অবশেষে আমরাও খেয়ে আসলাম Pizz...


In [11]:
def preprocess(sentence):
    sentence = re.sub("['।,]+",' ',sentence)
    sentence = re.sub(r'\n', '', sentence)
    sentence = re.sub(r'[ ]+', ' ', sentence)
    return sentence.strip().lower()

In [12]:
data.category_description = data.category_description.map(preprocess)
data.sample(5)

,Review,Label,category_description
3259,কাজে গিয়েছিলাম পান্থপথ। দুপুরে লাগে ক্ষুধা লাগ...,__label__Not_Fake,__label__not_fake কাজে গিয়েছিলাম পান্থপথ দুপুর...
2854,টেক আউট বা প্রিতমের সাথে কমপেয়ার করবোনা তবে মি...,__label__Not_Fake,__label__not_fake টেক আউট বা প্রিতমের সাথে কমপ...
427,#খানাপিনা_সমাচার♥️💃🏻\r\nখেতে গিয়েছিলাম ইন্টার...,__label__Fake,__label__fake #খানাপিনা_সমাচার♥️💃🏻\rখেতে গিয়ে...
1527,#CONFERMATION_POST....রিভিউ দেখে খাবারটা খেতে ...,__label__Not_Fake,__label__not_fake #confermation_post....রিভিউ ...
2997,The Pabulumবার্গারের অথেনটিক স্বাদ না পেলেও এ্...,__label__Not_Fake,__label__not_fake the pabulumবার্গারের অথেনটিক...


In [13]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [14]:
train_set.category_description

3255    __label__not_fake the pabulum #a_great_initiat...
2009    __label__not_fake #৯৯৳তান্দুরী আস্ত একটা মুরগি...
712                            __label__fake #paid review
2861    __label__not_fake কাচ্চি লাভারদের জন্য বর্তমান...
1506    __label__not_fake #১২১টাকা নাকি #২৬টাকা#২৬শে_ম...
                              ...                        
1095    __label__fake place: রাজমহল ফুডকোর্ট \rprice: ...
1130    __label__fake হ্যালো ভোজন প্রিয় নারায়ণগঞ্জ বা...
1294    __label__fake আচ্ছা! বাম চোখ লাফালে কি বিপদ আস...
860     __label__fake #brownie ❤ \rthis fudgy brownie ...
3174    __label__not_fake আনন্দময় ঈদের কাছাকাছি সময়ে র...
Name: category_description, Length: 2671, dtype: object

In [15]:
train_set.to_csv('food_review.train',columns=["category_description"],index=False,header=False)
test_set.to_csv('food_review.test',columns=["category_description"],index=False,header=False)

In [16]:
model = fasttext.train_supervised(input='food_review.train')


In [17]:
model.test("food_review.test")

(351, 0.9772079772079773, 0.9772079772079773)

In [18]:
model.predict('''○বান্ধুবিরা মিলে অনেক দিন পর বের হয়েছিলাম রিক্সায় ঘুরতে কিন্তুু ওরা খালি চাস্ন থাকে treat জন্য আমার আবার সুন্দর সুন্দর sun glass পরতে অনেক ভাল লাগে নতুন একটা চশমা পরে আজকে বের হয়েছিলাম বান্ধুবিরাও বললো েতাকে মানাইসে treat বাহানা আর খুইজা না পাওয়ায় বলতাসে নতুন glass কিনসস সেই খুশিতে treat দে 😒 কি আর করার অই দিন ফুড ব্যাংকে burger fiesta review দেক্লাম খুব reasonable price তাদের ফুডের আর খাবারও মজা আগেও তাদের ফুড আমি test করেছি 😋then burger fiesta গেলাম আমরা অডার করেছিলাম naga wings 🍗naga wings টা খুবই ঝাল ঝাল আমার খুব মজা লাগে জারা ঝাল খেতে পারেন must be try করবেন and সাথে অবশ্যই cold coffee নিতে হবে তাহলে test টা আরও মজা লাগে 😋 i love this burger fiesta s french fry 🍟🧡তাদের french fry টা খুবই crispy হয় and last one chicken sandwich as a good test 🥪 1.naga wings 4 pics/🍗99 tk. 2.chicken sandwich 4 pics🥪/99 tk. 3.french fry /100 tk 🍟 4.cold coffee 70 tk🍹''')

(('__label__not_fake',), array([0.91562426]))

In [19]:
model.save_model('bangla_food_review.model')